In [78]:
import numpy as np
import matplotlib.pyplot as plt
import timeit

from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri, default_converter
from rpy2.robjects.conversion import localconverter

glstudy = importr("glstudy")
np_cv_rules = default_converter + numpy2ri.converter

In [79]:
def check_acc(beta, L, v, l1, l2):
    return beta - (v /(L + l2 + l1 / np.sqrt(np.sum(beta**2))))

In [80]:
def bench(ps, configs):
    n_ps = len(ps)
    n_algs = 2
    index_names = ['Newton', 'FISTA']
    times = np.zeros((n_algs, n_ps))
    iters = np.zeros((n_algs, n_ps))
    accs = np.zeros((n_algs, n_ps))
    
    seed = configs['seed']
    max_eval = configs['max_eval']
    l1 = configs['l1']
    l2 = configs['l2']
    newton_tol = configs['newton_tol']
    fista_tol = configs['fista_tol']
    max_iters = configs['max_iters']

    for i, p in enumerate(ps):
        np.random.seed(seed + 100*i)
        L = np.random.uniform(0, max_eval, size=(p,))
        v = np.random.normal(0, 1, size=(p,)) * np.sqrt(L)

        # benchmark 
        with localconverter(np_cv_rules):
            newton_times = timeit.repeat(
                lambda: glstudy.newton_solver(L, v, l1, l2, newton_tol, max_iters), 
                repeat=1, 
                number=10,
            )
            fista_times = timeit.repeat(
                lambda: glstudy.fista_solver(L, v, l1, l2, fista_tol, max_iters), 
                repeat=1, 
                number=10,
            )
        
            # call one more time for outputs
            newton_out = glstudy.newton_solver(L, v, l1, l2, newton_tol, max_iters) 
            fista_out = glstudy.fista_solver(L, v, l1, l2, fista_tol, max_iters)

            # save output
            times[:, i] = [
                np.mean(newton_times), 
                np.mean(fista_times),
            ]
            iters[:, i] = [
                newton_out['iters'][0],
                fista_out['iters'][0],
            ]
            accs[:, i] =[
                np.max(np.abs(check_acc(newton_out['beta'], L, v, l1, l2))),
                np.max(np.abs(check_acc(fista_out['beta'], L, v, l1, l2))),
            ]

    return index_names, times, iters, accs


def vbench(configs):
    return [bench(**config) for config in configs]


In [81]:
ps = np.array([10, 100, 1000])
bench(ps, {'seed': 0, 'max_eval': 10, 'l1': 1, "l2": 0, "newton_tol": 1e-8, "fista_tol": 1e-14, "max_iters": int(1e5)})

(['Newton', 'FISTA'],
 array([[0.0045207 , 0.00275121, 0.00315107],
        [0.00296796, 0.00449971, 0.06003289]]),
 array([[5.000e+00, 5.000e+00, 6.000e+00],
        [8.700e+01, 1.821e+03, 5.899e+03]]),
 array([[5.92859095e-14, 5.07111020e-10, 2.50075516e-11],
        [8.43769499e-15, 2.53908006e-13, 8.19206925e-11]]))